In [1]:
DELAY = 0.1

In [2]:
# from websocket import create_connection
import asyncio
import websockets
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from time import sleep

In [3]:
# data = pd.read_csv("../data/IoT_Garage_Door.csv", low_memory=False)
data = pd.read_csv("../data/test_garage.csv", low_memory=False)

In [4]:
data = data.dropna()
encoder=LabelEncoder()

# For IoT_Garade_Door
data['type']=encoder.fit_transform(data['type'])
data['state']=encoder.fit_transform(data['state'])
data['sphone_signal']=encoder.fit_transform(data['sphone_signal'])

In [5]:
data.head()

,date,time,state,sphone_signal,label,type
0,1-Apr-19,20:53:44,1,1,0,3
1,1-Apr-19,20:53:49,0,0,0,3
2,1-Apr-19,20:53:49,1,1,0,3
3,1-Apr-19,20:53:54,0,0,0,3
4,1-Apr-19,20:53:54,1,1,0,3


In [6]:
async def server(websocket):
    temp = data.to_string(header=False,index=False,
                  index_names=False).split('\n')
    rows = [','.join(ele.split()) for ele in temp]
    # row = await websocket.recv()
    c=0
    while(len(data)>=c):
        await websocket.send(rows[c])
        ok = await websocket.recv()
        if(bool(ok)):
            c+=1
        sleep(DELAY)

async def main():
    async with websockets.serve(server, "localhost", 8765, ping_interval=None):
        await asyncio.Future()  # run forever

await main()

connection handler failed
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/websockets/legacy/server.py", line 232, in handler
    await self.ws_handler(self)
  File "/tmp/ipykernel_6030/3810110518.py", line 8, in server
    await websocket.send(rows[c])
  File "/usr/local/lib/python3.10/dist-packages/websockets/legacy/protocol.py", line 620, in send
    await self.ensure_open()
  File "/usr/local/lib/python3.10/dist-packages/websockets/legacy/protocol.py", line 930, in ensure_open
    raise self.connection_closed_exc()
websockets.exceptions.ConnectionClosedOK: sent 1001 (going away); then received 1001 (going away)


CancelledError: 